In [1]:
import pandas
import tensorflow as tf
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

C:\Users\Public\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
data = pd.read_csv('California_Housing_Data.csv')

In [33]:
data.head()

,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedInc,medianHouseValue
0,41.0,6.984127,1.023810,322.0,2.555556,8.3252,4.526
1,21.0,6.238137,0.971880,2401.0,2.109842,8.3014,3.585
2,52.0,8.288136,1.073446,496.0,2.802260,7.2574,3.521
3,52.0,5.817352,1.073059,558.0,2.547945,5.6431,3.413
4,52.0,6.281853,1.081081,565.0,2.181467,3.8462,3.422


In [7]:
data.drop(labels=['Unnamed: 0','Longitude', 'Latitude'], inplace=True, axis=1)

In [8]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
HouseAge,20640.0,28.639486,12.585558,1.000000,18.000000,29.000000,37.000000,52.000000
AveRooms,20640.0,5.429000,2.474173,0.846154,4.440716,5.229129,6.052381,141.909091
AveBedrms,20640.0,1.096675,0.473911,0.333333,1.006079,1.048780,1.099526,34.066667
Population,20640.0,1425.476744,1132.462122,3.000000,787.000000,1166.000000,1725.000000,35682.000000
AveOccup,20640.0,3.070655,10.386050,0.692308,2.429741,2.818116,3.282261,1243.333333
MedInc,20640.0,3.870671,1.899822,0.499900,2.563400,3.534800,4.743250,15.000100
medianHouseValue,20640.0,2.068558,1.153956,0.149990,1.196000,1.797000,2.647250,5.000010


In [9]:
y_data = data['medianHouseValue']
x_data = data.drop(['medianHouseValue'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.3)

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

In [13]:
data.columns

Index(['HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'MedInc',
       'medianHouseValue'],
      dtype='object')

In [49]:
age = tf.feature_column.numeric_column('HouseAge')
rooms = tf.feature_column.numeric_column('AveRooms')
bedroom = tf.feature_column.numeric_column('AveBedrms')
population = tf.feature_column.numeric_column('Population')
households = tf.feature_column.numeric_column('AveOccup')
income = tf.feature_column.numeric_column('MedInc')


In [50]:
fcols = [age, rooms, bedroom, population, households, income]

In [21]:
input_function  = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

In [39]:
model = tf.estimator.DNNRegressor(hidden_units=[4,4,4,4,4],feature_columns=fcols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\20D3~1\\AppData\\Local\\Temp\\tmp_8yi9297', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000001012C710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [40]:
model.train(input_fn = input_function,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\20D3~1\AppData\Local\Temp\tmp_8yi9297\model.ckpt.
INFO:tensorflow:loss = 66.95183, step = 1
INFO:tensorflow:global_step/sec: 196.067
INFO:tensorflow:loss = 4.241087, step = 101 (0.522 sec)
INFO:tensorflow:global_step/sec: 270.987
INFO:tensorflow:loss = 11.188167, step = 201 (0.368 sec)
INFO:tensorflow:global_step/sec: 270.255
INFO:tensorflow:loss = 4.2911143, step = 301 (0.374 sec)
INFO:tensorflow:global_step/sec: 241.532
INFO:tensorflow:loss = 14.714896, step = 401 (0.403 sec)
INFO:tensorflow:global_step/sec: 270.987
INFO:tensorflow:loss = 2.3015184, step = 501 (0.392 sec)
INFO:tensorflow:global_step/sec: 217.853
INFO:tensorflow:loss = 15.507094, step = 601 (0.441 sec)
INFO:tensorflow:global_step/s

In [41]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=10, num_epochs=1, shuffle=False)

In [42]:
preds = model.predict(predict_input_func)

In [43]:
predictions = list(preds)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\20D3~1\AppData\Local\Temp\tmp_8yi9297\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [44]:
final_pred = []
for pred in predictions:
    final_pred.append(pred['predictions'])

In [45]:
final_pred

[array([1.3906044], dtype=float32),
 array([4.672939], dtype=float32),
 array([1.9892285], dtype=float32),
 array([2.7875416], dtype=float32),
 array([1.2156211], dtype=float32),
 array([1.0900899], dtype=float32),
 array([1.7998496], dtype=float32),
 array([1.6114104], dtype=float32),
 array([2.658327], dtype=float32),
 array([1.3350666], dtype=float32),
 array([1.8043776], dtype=float32),
 array([1.6371257], dtype=float32),
 array([0.73750144], dtype=float32),
 array([3.5282323], dtype=float32),
 array([1.2082344], dtype=float32),
 array([1.0273695], dtype=float32),
 array([1.6117716], dtype=float32),
 array([1.5447383], dtype=float32),
 array([2.7249327], dtype=float32),
 array([1.5504762], dtype=float32),
 array([1.2974546], dtype=float32),
 array([1.8456689], dtype=float32),
 array([1.7070965], dtype=float32),
 array([1.2138417], dtype=float32),
 array([1.9623798], dtype=float32),
 array([2.1108139], dtype=float32),
 array([3.0515723], dtype=float32),
 array([1.3846501], dtype=flo

In [46]:
result = math.sqrt(mean_squared_error(y_test,final_pred))
print('RMSE:', result)

RMSE: 0.7998278755533678
